In [1]:
import pandas as pd

In [2]:
# unicef data WH
wh_url = "https://sdmx.data.unicef.org/ws/public/sdmxapi/rest/data/"
dataflow = "TRANSMONEE/"

In [3]:
# indicators pulled
ind_list = [
    "EDUNF_OFST_L1T3",
    "EDUNF_ROFST_L1T3",
    "EDUNF_SAP_L1T3",
    "EDUNF_OFST_L1_UNDER1",
    "EDUNF_OFST_L1",
    "EDUNF_OFST_L2",
    "EDUNF_OFST_L3",
    "EDUNF_NER_L02",
    "EDUNF_SAP_L02",
    "EDUNF_CR_L2",
    "EDUNF_SAP_L2",
    "EDUNF_ESL_L1",
    "EDUNF_RPTR_L1",
    "EDUNF_RPTR_L2",
    "EDUNF_ADMIN_L1_GLAST_REA",
    "EDUNF_ADMIN_L1_GLAST_MAT",
    "EDU_PISA_MAT",
    "EDU_PISA_REA",
    "EDU_PISA_SCI",
    "EDU_SDG_FREE_EDU_L02",
    "EDUNF_STU_L1_PRV",
    "EDUNF_STU_L2_PRV",
    "EDUNF_STU_L3_PRV",
    "EDUNF_TEACH_L1",
    "EDUNF_TEACH_L2",
    "EDUNF_TEACH_L3",
]

In [4]:
# query SDMX with pandas
query_key = ".{}....".format("+".join(ind_list))
data = pd.read_csv(wh_url + dataflow + query_key + "?format=csv")

In [5]:
# set countries and period
countries = ["Armenia", "Azerbaijan", "Georgia"]
time = list(range(2008,2021))
total_code = ["_T"]

In [6]:
# calculation single numerator
numerator = ["EDUNF_OFST_L1T3"]
denominator = ["EDUNF_SAP_L1T3"]

In [7]:
query = (
        "INDICATOR in @ind_to_query & TIME_PERIOD in @time & `Geographic area` in @countries & SEX in @total_code \
& RESIDENCE in @total_code & WEALTH_QUINTILE in @total_code"
    )

In [8]:
ind_to_query = numerator
numerators_df = data.query(query).groupby(
    [
        "INDICATOR",
        "Indicator",
        "Geographic area"
    ]
).agg(
    {"TIME_PERIOD": "last", "OBS_VALUE": "last"}
).reset_index().set_index(["Geographic area", "TIME_PERIOD"])
numerators_df

,,INDICATOR,Indicator,OBS_VALUE
Geographic area,TIME_PERIOD,,,
Armenia,2011,EDUNF_OFST_L1T3,"Number of out-of-school children, adolescents ...",29523.0
Azerbaijan,2018,EDUNF_OFST_L1T3,"Number of out-of-school children, adolescents ...",32589.0
Georgia,2018,EDUNF_OFST_L1T3,"Number of out-of-school children, adolescents ...",11266.0


In [9]:
# select the denominators and set them with numerator compound-index
ind_to_query = denominator
denominators_df = data.query(query).set_index(["Geographic area", "TIME_PERIOD"])
# match denominators and numerators compound-index
den_match_df = denominators_df[
    denominators_df.index.isin(numerators_df.index)
][["INDICATOR", "Indicator", "OBS_VALUE"]]
den_match_df

,,INDICATOR,Indicator,OBS_VALUE
Geographic area,TIME_PERIOD,,,
Armenia,2011,EDUNF_SAP_L1T3,Population of the official age for primary and...,413525.0
Azerbaijan,2018,EDUNF_SAP_L1T3,Population of the official age for primary and...,1637373.0
Georgia,2018,EDUNF_SAP_L1T3,Population of the official age for primary and...,574212.0


In [10]:
# join numerators denominators with nan drops (missmatches)
num_den_not_nan = numerators_df.merge(
    den_match_df, on=["Geographic area", "TIME_PERIOD"], how="left", sort=False
).dropna(subset=["OBS_VALUE_y"])
num_den_not_nan

,,INDICATOR_x,Indicator_x,OBS_VALUE_x,INDICATOR_y,Indicator_y,OBS_VALUE_y
Geographic area,TIME_PERIOD,,,,,,
Armenia,2011,EDUNF_OFST_L1T3,"Number of out-of-school children, adolescents ...",29523.0,EDUNF_SAP_L1T3,Population of the official age for primary and...,413525.0
Azerbaijan,2018,EDUNF_OFST_L1T3,"Number of out-of-school children, adolescents ...",32589.0,EDUNF_SAP_L1T3,Population of the official age for primary and...,1637373.0
Georgia,2018,EDUNF_OFST_L1T3,"Number of out-of-school children, adolescents ...",11266.0,EDUNF_SAP_L1T3,Population of the official age for primary and...,574212.0


In [11]:
print((29523.0+32589.0+11266.0) / (413525.0 + 1637373.0 + 574212.0)*100)

2.7952352472848756


In [12]:
# calculation with join
round(num_den_not_nan["OBS_VALUE_x"].sum()/num_den_not_nan["OBS_VALUE_y"].sum()*100,1)

2.8

In [13]:
ind_intersect = denominators_df.index.intersection(numerators_df.index)
ind_intersect

MultiIndex([(   'Armenia', 2011),
            ('Azerbaijan', 2018),
            (   'Georgia', 2018)],
           names=['Geographic area', 'TIME_PERIOD'])

In [14]:
# calculation without join
round(numerators_df.loc[ind_intersect]["OBS_VALUE"].sum()/denominators_df.loc[ind_intersect]["OBS_VALUE"].sum()*100,1)

2.8

In [15]:
# calculation single numerator
numerator = ["EDUNF_OFST_L1_UNDER1"]
ind_to_query = numerator
numerators_df = data.query(query).groupby(
    [
        "INDICATOR",
        "Indicator",
        "Geographic area"
    ]
).agg(
    {"TIME_PERIOD": "last", "OBS_VALUE": "last"}
).reset_index().set_index(["Geographic area", "TIME_PERIOD"])
numerators_df

,,INDICATOR,Indicator,OBS_VALUE
Geographic area,TIME_PERIOD,,,
Armenia,2018,EDUNF_OFST_L1_UNDER1,Number of out-of-school children one year youn...,22591.0
Azerbaijan,2018,EDUNF_OFST_L1_UNDER1,Number of out-of-school children one year youn...,53196.0


In [16]:
# calculation single numerator
numerator = ["EDUNF_ESL_L1"]
ind_to_query = numerator
numerators_df = data.query(query).groupby(
    [
        "INDICATOR",
        "Indicator",
        "Geographic area"
    ]
).agg(
    {"TIME_PERIOD": "last", "OBS_VALUE": "last"}
).reset_index().set_index(["Geographic area", "TIME_PERIOD"])
numerators_df

,,INDICATOR,Indicator,OBS_VALUE
Geographic area,TIME_PERIOD,,,
Armenia,2018,EDUNF_ESL_L1,Number of early school leavers from primary ed...,2176.58087
Azerbaijan,2018,EDUNF_ESL_L1,Number of early school leavers from primary ed...,9209.62321
Georgia,2018,EDUNF_ESL_L1,Number of early school leavers from primary ed...,591.04149


In [17]:
# calculation single numerator
numerator = ["EDUNF_ADMIN_L1_GLAST_REA"]
ind_to_query = numerator
numerators_df = data.query(query).groupby(
    [
        "INDICATOR",
        "Indicator",
        "Geographic area"
    ]
).agg(
    {"TIME_PERIOD": "last", "OBS_VALUE": "last"}
).reset_index().set_index(["Geographic area", "TIME_PERIOD"])
numerators_df

,,INDICATOR,Indicator,OBS_VALUE
Geographic area,TIME_PERIOD,,,
Armenia,2018,EDUNF_ADMIN_L1_GLAST_REA,Administration of nationally representative le...,0.0
Azerbaijan,2018,EDUNF_ADMIN_L1_GLAST_REA,Administration of nationally representative le...,1.0
Georgia,2018,EDUNF_ADMIN_L1_GLAST_REA,Administration of nationally representative le...,0.0


In [18]:
# calculation single numerator
numerator = ["EDUNF_ADMIN_L1_GLAST_MAT"]
ind_to_query = numerator
numerators_df = data.query(query).groupby(
    [
        "INDICATOR",
        "Indicator",
        "Geographic area"
    ]
).agg(
    {"TIME_PERIOD": "last", "OBS_VALUE": "last"}
).reset_index().set_index(["Geographic area", "TIME_PERIOD"])
numerators_df

,,INDICATOR,Indicator,OBS_VALUE
Geographic area,TIME_PERIOD,,,
Armenia,2018,EDUNF_ADMIN_L1_GLAST_MAT,Administration of nationally representative le...,1.0
Azerbaijan,2018,EDUNF_ADMIN_L1_GLAST_MAT,Administration of nationally representative le...,0.0
Georgia,2018,EDUNF_ADMIN_L1_GLAST_MAT,Administration of nationally representative le...,0.0


In [19]:
# calculation single numerator
numerator = ["EDU_SDG_FREE_EDU_L02"]
ind_to_query = numerator
numerators_df = data.query(query).groupby(
    [
        "INDICATOR",
        "Indicator",
        "Geographic area"
    ]
).agg(
    {"TIME_PERIOD": "last", "OBS_VALUE": "last"}
).reset_index().set_index(["Geographic area", "TIME_PERIOD"])
numerators_df

,,INDICATOR,Indicator,OBS_VALUE
Geographic area,TIME_PERIOD,,,
Armenia,2018,EDU_SDG_FREE_EDU_L02,Number of years of free pre-primary education ...,3.0
Azerbaijan,2018,EDU_SDG_FREE_EDU_L02,Number of years of free pre-primary education ...,5.0
Georgia,2018,EDU_SDG_FREE_EDU_L02,Number of years of free pre-primary education ...,0.0


In [20]:
# calculation with a "pack" of numerators
numerator = ["EDUNF_OFST_L1", "EDUNF_OFST_L2", "EDUNF_OFST_L3"]
ind_to_query = numerator
numerators_df = data.query(query).groupby(
    [
        "Geographic area",
        "TIME_PERIOD"
    ]
).agg(
    {"OBS_VALUE": "sum", "DATA_SOURCE": "count"}
).reset_index()
numerator_pairs = numerators_df[numerators_df.DATA_SOURCE == len(ind_to_query)].groupby(
    "Geographic area", as_index=False
).last().set_index(["Geographic area", "TIME_PERIOD"])
numerator_pairs

,,OBS_VALUE,DATA_SOURCE
Geographic area,TIME_PERIOD,,
Armenia,2011,29523.0,3
Azerbaijan,2018,32589.0,3
Georgia,2018,11265.0,3


In [21]:
# regional average using "pack" of numerators and one denominator
round(numerator_pairs.loc[ind_intersect]["OBS_VALUE"].sum() / 
      denominators_df.loc[ind_intersect]["OBS_VALUE"].sum()*100,1)

2.8

In [22]:
# calculation with a "pack" of numerators (REDONE FOR SEX FEMALE)
sex_code = ["F"]
query = (
        "INDICATOR in @ind_to_query & TIME_PERIOD in @time & `Geographic area` in @countries & SEX in @sex_code \
& RESIDENCE in @total_code & WEALTH_QUINTILE in @total_code"
    )
numerator = ["EDUNF_OFST_L1", "EDUNF_OFST_L2", "EDUNF_OFST_L3"]
ind_to_query = numerator
numerators_df = data.query(query).groupby(
    [
        "Geographic area",
        "TIME_PERIOD"
    ]
).agg(
    {"OBS_VALUE": "sum", "DATA_SOURCE": "count"}
).reset_index()
numerator_pairs = numerators_df[numerators_df.DATA_SOURCE == len(ind_to_query)].groupby(
    "Geographic area", as_index=False
).last().set_index(["Geographic area", "TIME_PERIOD"])
numerator_pairs

,,OBS_VALUE,DATA_SOURCE
Geographic area,TIME_PERIOD,,
Armenia,2011,12799.0,3
Georgia,2014,8205.0,3


In [23]:
# calculation with a "pack" of numerators (repeaters)
sex_code = ["_T"]
numerator = ["EDUNF_RPTR_L1","EDUNF_RPTR_L2"]
ind_to_query = numerator
numerators_df = data.query(query).groupby(
    [
        "Geographic area",
        "TIME_PERIOD"
    ]
).agg(
    {"OBS_VALUE": "sum", "DATA_SOURCE": "count"}
).reset_index()
numerator_pairs = numerators_df[numerators_df.DATA_SOURCE == len(ind_to_query)].groupby(
    "Geographic area", as_index=False
).last().set_index(["Geographic area", "TIME_PERIOD"])
numerator_pairs

,,OBS_VALUE,DATA_SOURCE
Geographic area,TIME_PERIOD,,
Armenia,2018,763.0,2
Azerbaijan,2018,2588.0,2
Georgia,2018,2736.0,2


In [24]:
# calculation with a "pack" of numerators (PISA)
numerator = ["EDU_PISA_MAT","EDU_PISA_REA","EDU_PISA_SCI"]
ind_to_query = numerator
numerators_df = data.query(query).groupby(
    [
        "Geographic area",
        "TIME_PERIOD"
    ]
).agg(
    {"OBS_VALUE": "sum", "DATA_SOURCE": "count"}
).reset_index()
# check all PISA not only those appearing for all "pack" of numerators
numerator_pairs = numerators_df.groupby("Geographic area", as_index=False).last().set_index(["Geographic area", "TIME_PERIOD"])
numerator_pairs

,,OBS_VALUE,DATA_SOURCE
Geographic area,TIME_PERIOD,,
Azerbaijan,2009,1165.674206,3
Georgia,2018,1160.002502,3


In [25]:
# calculation with a "pack" of numerators (private enrolments)
numerator = ["EDUNF_STU_L1_PRV","EDUNF_STU_L2_PRV","EDUNF_STU_L3_PRV"]
ind_to_query = numerator
numerators_df = data.query(query).groupby(
    [
        "Geographic area",
        "TIME_PERIOD"
    ]
).agg(
    {"OBS_VALUE": "sum", "DATA_SOURCE": "count"}
).reset_index()
# check all PISA not only those appearing for all "pack" of numerators
numerator_pairs = numerators_df.groupby("Geographic area", as_index=False).last().set_index(["Geographic area", "TIME_PERIOD"])
numerator_pairs

,,OBS_VALUE,DATA_SOURCE
Geographic area,TIME_PERIOD,,
Armenia,2018,9353.0,3
Azerbaijan,2018,126300.0,3
Georgia,2018,60062.0,3


In [26]:
# calculation with a "pack" of numerators (private enrolments)
numerator = ["EDUNF_TEACH_L1", "EDUNF_TEACH_L2", "EDUNF_TEACH_L3"]
ind_to_query = numerator
numerators_df = data.query(query).groupby(
    [
        "Geographic area",
        "TIME_PERIOD"
    ]
).agg(
    {"OBS_VALUE": "sum", "DATA_SOURCE": "count"}
).reset_index()
# check all PISA not only those appearing for all "pack" of numerators
numerator_pairs = numerators_df.groupby("Geographic area", as_index=False).last().set_index(["Geographic area", "TIME_PERIOD"])
numerator_pairs

,,OBS_VALUE,DATA_SOURCE
Geographic area,TIME_PERIOD,,
Armenia,2018,39261.0,3
Azerbaijan,2018,165375.0,3
Georgia,2018,71016.0,3


In [27]:
# check the call with rates! (and only one numerator in the list)
numerator = ["EDUNF_NER_L02"]
ind_to_query = numerator
numerators_df = data.query(query).groupby(
    [
        "Geographic area",
        "TIME_PERIOD"
    ]
).agg(
    {"OBS_VALUE": "sum", "DATA_SOURCE": "count"}
).reset_index()
numerator_pairs = numerators_df[numerators_df.DATA_SOURCE == len(ind_to_query)].groupby(
    "Geographic area", as_index=False
).last().set_index(["Geographic area", "TIME_PERIOD"])
numerator_pairs

,,OBS_VALUE,DATA_SOURCE
Geographic area,TIME_PERIOD,,
Armenia,2018,35.14269,1
Azerbaijan,2018,35.36377,1


In [28]:
denominator = ["EDUNF_SAP_L02"]
# select the denominators and set them with numerator compound-index
ind_to_query = denominator
denominators_df = data.query(query).set_index(["Geographic area", "TIME_PERIOD"])

In [29]:
# select only those denominators that match avalible indicators
ind_intersect = numerator_pairs.index.intersection(
    denominators_df.index)

In [30]:
absolute = False
denominators = denominators_df.loc[ind_intersect]["OBS_VALUE"]
indicator_sum = (
    numerator_pairs.loc[ind_intersect]["OBS_VALUE"].to_numpy().sum(
    )*100 / denominators.to_numpy().sum()
    if absolute
    else (
        numerator_pairs["OBS_VALUE"] * denominators / denominators.to_numpy().sum()
    ).dropna().to_numpy().sum()
)
# will drop missing countires
round(indicator_sum,1)

35.3

In [31]:
# check it out BETO!
denominators

Geographic area  TIME_PERIOD
Armenia          2018           129438.0
Azerbaijan       2018           511204.0
Name: OBS_VALUE, dtype: float64

In [32]:
print((35.14269*129438.0 + 35.36377*511204.0) / (129438.0 + 511204.0))
print((35.14269*129438.0 + 35.36377*511204.0 + 40.53383*137347.0) / (129438.0 + 511204.0 + 137347.0))

35.319102068393896
36.23971564676365


In [33]:
# check the call with rates! (and only one numerator in the list)
numerator = ["EDUNF_CR_L2"]
ind_to_query = numerator
numerators_df = data.query(query).groupby(
    [
        "Geographic area",
        "TIME_PERIOD"
    ]
).agg(
    {"OBS_VALUE": "sum", "DATA_SOURCE": "count"}
).reset_index()
numerator_pairs = numerators_df[numerators_df.DATA_SOURCE == len(ind_to_query)].groupby(
    "Geographic area", as_index=False
).last().set_index(["Geographic area", "TIME_PERIOD"])
numerator_pairs

,,OBS_VALUE,DATA_SOURCE
Geographic area,TIME_PERIOD,,
Armenia,2016,96.96779,1
Georgia,2013,98.65000,1


In [34]:
denominator = ["EDUNF_SAP_L2"]
# select the denominators and set them with numerator compound-index
ind_to_query = denominator
denominators_df = data.query(query).set_index(["Geographic area", "TIME_PERIOD"])

In [35]:
# select only those denominators that match avalible indicators
ind_intersect = numerator_pairs.index.intersection(
    denominators_df.index)

In [36]:
absolute = False
denominators = denominators_df.loc[ind_intersect]["OBS_VALUE"]
indicator_sum = (
    numerator_pairs.loc[ind_intersect]["OBS_VALUE"].to_numpy().sum(
    )*100 / denominators.to_numpy().sum()
    if absolute
    else (
        numerator_pairs["OBS_VALUE"] * denominators / denominators.to_numpy().sum()
    ).dropna().to_numpy().sum()
)
# will drop missing countires
round(indicator_sum,4)

97.7257

In [37]:
# check it out BETO!
denominators

Geographic area  TIME_PERIOD
Armenia          2016           173593.0
Georgia          2013           142350.0
Name: OBS_VALUE, dtype: float64

In [38]:
print((96.96779*173593.0 + 98.65000*142350.0) / (173593.0 + 142350.0))
print((96.96779*173593.0 + 98.65000*142350.0 + 91.65569*868438.0) / (173593.0 + 142350.0 + 868438.0))

97.72571973257834
93.27491844405644


In [39]:
# actual function implemented in base page DASH
def indicator_card(
    time,
    countries,
    numerator,
    suffix,
    denominator=None,
    absolute=False,
    sex_code = ["_T"]
):
    # sex_code = ["_T"]  # potentially move to this config
    total_code = ["_T"]  # potentially move to this config
    query = "CODE in @indicator & TIME_PERIOD in @time & `Geographic area` in @countries & SEX in @sex_code \
        & RESIDENCE in @total_code & WEALTH_QUINTILE in @total_code"
    numors = numerator.split(",")
    indicator = numors
    # select last value for each country
    indicator_values = (
        data.query(query)
        .groupby(
            [
                "Geographic area",
                "TIME_PERIOD",
            ]
        )
        .agg({"OBS_VALUE": "sum", "DATA_SOURCE": "count"})
    ).reset_index()

    numerator_pairs = (
        indicator_values[indicator_values.DATA_SOURCE == len(numors)]
        .groupby("Geographic area", as_index=False)
        .last()
        .set_index(["Geographic area", "TIME_PERIOD"])
    )

    # check for denominator
    if denominator:
        # select the avalible denominators for countiries in selected years
        indicator = [denominator]
        denominator_values = data.query(query).set_index(["Geographic area", "TIME_PERIOD"])
        # select only those denominators that match avalible indicators
        index_intersect = numerator_pairs.index.intersection(denominator_values.index)

        denominators = denominator_values.loc[index_intersect]["OBS_VALUE"]
    

        indicator_sum = (
            numerator_pairs.loc[index_intersect]["OBS_VALUE"].to_numpy().sum()
            / denominators.to_numpy().sum()
            * 100
            if absolute
            else (
                numerator_pairs["OBS_VALUE"] * denominators / denominators.to_numpy().sum()
            )
            .dropna()  # will drop missing countires
            .to_numpy()
            .sum()
        )
        sources = index_intersect.tolist()

    elif suffix == "Countries":
        # this is a hack to accomodate small cases (to discuss with James)
        if "FREE" in numerator:
            # trick to filter number of years of free education
            indicator_sum = (numerator_pairs.OBS_VALUE >= 1).to_numpy().sum()
            sources = numerator_pairs.index.tolist()
        elif absolute:
            # trick to accomodate cards for data availability among group of indicators
            # doesn't require filtering by count == len(numors)
            numerator_pairs = indicator_values.groupby("Geographic area", as_index=False).last().set_index(["Geographic area", "TIME_PERIOD"])
            sources = numerator_pairs.index.tolist()
            indicator_sum = len(sources)
        else:
            # trick to accomodate cards for admin exams (AND for boolean indicators)
            # filter exams according to number of indicators
            indicator_sum = (numerator_pairs.OBS_VALUE == len(numors)).to_numpy().sum()
            sources = numerator_pairs.index.tolist()

    else:
        indicator_sum = numerator_pairs["OBS_VALUE"].to_numpy().sum()
        sources = numerator_pairs.index.tolist()

    return ("{:.0f} {}".format(indicator_sum, suffix), sources)

In [40]:
# rename INDICATOR as CODE as in DASH init.py
data.rename(columns={"INDICATOR": "CODE"}, inplace=True)
# test and compare function
countries = ["Armenia", "Azerbaijan", "Georgia"]
time = list(range(2008,2021))
numerator = "EDUNF_OFST_L1,EDUNF_OFST_L2,EDUNF_OFST_L3"
# numerator = "EDUNF_OFST_L1T3"
denominator = "EDUNF_SAP_L1T3"
# numerator = "EDUNF_CR_L2"
# denominator = "EDUNF_SAP_L2"
# numerator = "EDUNF_NER_L02"
# denominator = "EDUNF_SAP_L02"
suffix = "%"
absolute = True
a, b = indicator_card(time,countries,numerator,suffix,denominator,absolute)
print(a)
print(b)

3 %
[('Armenia', 2011), ('Azerbaijan', 2018), ('Georgia', 2018)]


In [41]:
# Test of Cards Version 2
participation = [
    {
        "name": "Total Number OOS (P+LS+US)",
        "indicator": "EDUNF_OFST_L1,EDUNF_OFST_L2,EDUNF_OFST_L3",
        "suffix": "Children, Adolescents and Youth",
    },
    {
        "name": "Number of Girls OOS (P+LS+US)",
        "indicator": "EDUNF_OFST_L1,EDUNF_OFST_L2,EDUNF_OFST_L3",
        "suffix": "Female Children, Adolescents and Youth",
        "sex": "F",
    },
    {
        "name": "Total Number OOS Under-1 Year (P)",
        "indicator": "EDUNF_OFST_L1_UNDER1",
        "suffix": "Children under-1 year of the primary official entry age",
    },
]

In [42]:
numerator = "EDUNF_OFST_L1,EDUNF_OFST_L2,EDUNF_OFST_L3"
suffix = "Children, Adolescents and Youth"
a, b= indicator_card(time,countries,numerator,suffix)
print(a)
print(b)

73377 Children, Adolescents and Youth
[('Armenia', 2011), ('Azerbaijan', 2018), ('Georgia', 2018)]


In [43]:
29523.0	+ 32589.0 + 11265.0

73377.0

In [44]:
numerator = "EDUNF_OFST_L1,EDUNF_OFST_L2,EDUNF_OFST_L3"
suffix = "Children, Adolescents and Youth"
sex = ["F"]
a, b= indicator_card(time,countries,numerator,suffix,sex_code=sex)
print(a)
print(b)

21004 Children, Adolescents and Youth
[('Armenia', 2011), ('Georgia', 2014)]


In [45]:
12799.0 + 8205.0

21004.0

In [46]:
numerator = "EDUNF_OFST_L1_UNDER1"
suffix = "Children"
a, b= indicator_card(time,countries,numerator,suffix)
print(a)
print(b)

75787 Children
[('Armenia', 2018), ('Azerbaijan', 2018)]


In [47]:
22591.0 + 53196.0

75787.0

In [48]:
quality =  [
    {
        "name": "Total Number of Repeaters (P+LS)",
        "indicator": "EDUNF_RPTR_L1,EDUNF_RPTR_L2",
        "suffix": "Children and Adolescents",
    },
    {
        "name": "Total Number of Early School Leavers (P)",
        "indicator": "EDUNF_ESL_L1",
        "suffix": "Children",
    },
    {
        "name": "Number of Countries assessing both Math and Reading learning (End Primary)",
        "indicator": "EDUNF_ADMIN_L1_GLAST_REA,EDUNF_ADMIN_L1_GLAST_MAT",
        "suffix": "Countries",
    },
    {
        "name": "Number of Countries participating in PISA",
        "indicator": "EDU_PISA_MAT,EDU_PISA_REA,EDU_PISA_SCI",
        "suffix": "Countries",
        "absolute": True,
    },
]

In [49]:
numerator = "EDUNF_RPTR_L1,EDUNF_RPTR_L2"
suffix = "Children and Adolescents"
a, b= indicator_card(time,countries,numerator,suffix)
print(a)
print(b)

6087 Children and Adolescents
[('Armenia', 2018), ('Azerbaijan', 2018), ('Georgia', 2018)]


In [50]:
763.0 + 2588.0 + 2736.0

6087.0

In [51]:
numerator = "EDUNF_ESL_L1"
suffix = "Children"
a, b= indicator_card(time,countries,numerator,suffix)
print(a)
print(b)

11977 Children
[('Armenia', 2018), ('Azerbaijan', 2018), ('Georgia', 2018)]


In [52]:
2176.58087 + 9209.62321 + 591.04149

11977.24557

In [53]:
numerator = "EDUNF_ADMIN_L1_GLAST_REA,EDUNF_ADMIN_L1_GLAST_MAT"
suffix = "Countries"
a, b= indicator_card(time,countries,numerator,suffix)
print(a)
print(b)

0 Countries
[('Armenia', 2018), ('Azerbaijan', 2018), ('Georgia', 2018)]


In [54]:
numerator = "EDU_PISA_MAT,EDU_PISA_REA,EDU_PISA_SCI"
suffix = "Countries"
absolute = True
a, b= indicator_card(time,countries,numerator,suffix,absolute=absolute)
print(a)
print(b)

2 Countries
[('Azerbaijan', 2009), ('Georgia', 2018)]


In [55]:
governance = [
    {
        "name": "Number of Countries guaranteeing free education (Pre-Primary 1+ years)",
        "indicator": "EDU_SDG_FREE_EDU_L02",
        "suffix": "Countries",
    },
    {
        "name": "Enrolments in Private Institutions (P+LS+US)",
        "indicator": "EDUNF_STU_L1_PRV,EDUNF_STU_L2_PRV,EDUNF_STU_L3_PRV",
        "suffix": "Children, Adolescents and Youth",
    },
    {
        "name": "Total Number of classroom teachers (P+LS+US)",
        "indicator": "EDUNF_TEACH_L1,EDUNF_TEACH_L2,EDUNF_TEACH_L3",
        "suffix": "Persons",
    },
]

In [56]:
numerator = "EDU_SDG_FREE_EDU_L02"
suffix = "Countries"
a, b= indicator_card(time,countries,numerator,suffix)
print(a)
print(b)

2 Countries
[('Armenia', 2018), ('Azerbaijan', 2018), ('Georgia', 2018)]


In [57]:
numerator = "EDUNF_STU_L1_PRV,EDUNF_STU_L2_PRV,EDUNF_STU_L3_PRV"
suffix = "Children, Adolescents and Youth"
a, b= indicator_card(time,countries,numerator,suffix)
print(a)
print(b)

195715 Children, Adolescents and Youth
[('Armenia', 2018), ('Azerbaijan', 2018), ('Georgia', 2018)]


In [58]:
9353.0 + 126300.0 + 60062.0

195715.0

In [59]:
numerator = "EDUNF_TEACH_L1,EDUNF_TEACH_L2,EDUNF_TEACH_L3"
suffix = "Children, Adolescents and Youth"
a, b= indicator_card(time,countries,numerator,suffix)
print(a)
print(b)

275652 Children, Adolescents and Youth
[('Armenia', 2018), ('Azerbaijan', 2018), ('Georgia', 2018)]


In [60]:
39261.0 + 165375.0 + 71016.0

275652.0